# Demo for VirtualCity
##### This is a demo of how to VirtualCity. The demo will walk though how to start an environment and control it, how to prepare it to perform activities and finally how to perform activities in them.

## Setup
##### The first step is to start a communication with the simulator. Make sure before you run this notebook, that you have downloaded the correct version of the environment for your OS and your drivers and OS is updated to the latest version.

##### Install required packages

In [ ]:
# !pip install virtualcity
!pip install python-socketio
!pip install "python-socketio[client]"
!pip install eventlet

#### Communication server
##### This code is only run once to initialize the server. Run the following server code on a separate process (you can use tmux to run the server but avoid running it within the Jupyter Notebook file)

In [ ]:
import socketio
import eventlet
import threading

sio = socketio.Server(cors_allowed_origins="*")
app = socketio.WSGIApp(sio)

client_sids = []


@sio.event
def connect(sid, environ):
    print("Client connected:", sid)
    client_sids.append(sid)


@sio.event
def disconnect(sid):
    print("Client disconnected:", sid)
    client_sids.remove(sid)


@sio.event
def make(sid, data):
    print("Received make from {}: {}".format(sid, data))
    # Forward the message to all other connected clients (except the sender)
    for client_sid in client_sids:
        if client_sid != sid:
            sio.emit("make", data, room=client_sid)


def run_server():
    eventlet.wsgi.server(eventlet.listen(("", 5000)), app)


if __name__ == "__main__":
    thread = threading.Thread(target=run_server)
    thread.start()
    print("Server is running in background")

##### This code intialize the client communcation. It allows you to send commands to the server that you previously setup, which acts as a bridge between the executible file and the jupyter notebook. Run the following code to intialize the client communication with the environment

In [ ]:
import socketio

sio = socketio.Client()


@sio.event
def connect():
    print("Connected to server")


@sio.event
def disconnect():
    print("Disconnected from the server")


@sio.event
def message(data):
    print("Received message:", data)


sio.connect("http://localhost:5000")

### Import packages
##### Before sending commands to the environment, you can import the utils package with provides additional interactive functionality with the environment

In [ ]:

# import virtualcity
import utils

### Sending commands to the environment
##### When you want to start a environment or send a instruction to an agent, etc. You will need to pack the configuration in a dictionary and use the emit function to send the command to the environment. If the command was succesfully executed in the environment, you will recieve a message: {success: True} and if it failed then you will recieve a message: {sucess: False}. 

### Make a environment
##### To start a environment, you need to provide the environent name that you want to intialize. You can select from various prebuilt environments or use the "Procedural Generation" with a seed to create or recreate a previously generated environment on the fly.

In [ ]:
"""
Args:
    environment (str): The name of the environment. Default is 'Demo'.
    num_env (list): The number of environments. Default is [1].
"""
data = {
    "environment": "Demo", 
    # "num_env": [1]
}

sio.emit("make", data)

### Reset the environment
##### To reset a environment, you can choose to provide a environment graph to reset it to a previosly saved state. If no graph is provided, then the environment is reset to its default state. 

In [ ]:
"""
Args:
    env_index (list): The index of the environment. Default is [0].
    graph (dict): The environment graph in JSON. Default is None.
"""
import json

with open("graph.json", "r") as f:
    graph = json.load(f)

data = {
    "env_index": [0], 
    "graph": graph
}

sio.emit("reset", data)

### Create a graph observation
##### You can create a environment graph that describes the entire environment in a JSON file. The graph contains object information, relations, states, etc.

In [ ]:
"""
Args:
    type (str): The type of observation. Default is 'graph'.
    radius (list): The radius of the observation. Default is [-1].
"""
data = {
    "type": "graph", 
    # "radius": [-1]
}

sio.emit("observation", data)

### Set action for an agent
##### To send a action command to a agent, you can use provide a list containing the agent index, and a list describing the task which needs to be completed. The task list is structured in the following way: [action, object_1, relation, object_2]. To select a action with the appropriate object and relation, you can refer to the docs.

In [ ]:
"""
Args:
    agent_index (list): The index of the agent. Default is [0].
    task (list): The task assigned to the agent. Default is [0, 0, 0, 0].
    coordinate (list): The coordinate of the agent. Default is [0, 0, 0].
"""
data = {
    "agent_index": [1],
    "task": [1, 1, 0, 0],
    # "coordinate": coordinate
}

sio.emit("set_action", data)

##### A more interactive way is to create list of actions in natural language, and run it with the sequence function from the optional utils package. Try to follow a similar syntax as the example provided below when running your own unique scripts.

In [ ]:
import utils

# Sequence
action_list = [
    "agent_1 pickup object1",
    "agent_2 walk to object_2",
    "agent_3 place object_4 on object_5",
    "agent_4 jump over object_6",
    "agent_5 examine object3 under object_7",
    "agent_6 run from object_8 to object_9",
    "agent_7 pick object_10 and place on object_11",
]

utils.sequence(action_list)

### Change rendering settings
##### You can also change the rendering settings of the environment in real time such as the camera, resolution, fps, etc. 

In [ ]:
"""
Args:
    camera_index (list): The index of the camera. Default is [1].
    image_synthesis (str): Whether to use image synthesis. Default is raytracing.
    image_width (list): The width of the image. Default is [1920].
    image_height (list): The height of the image. Default is [480].
    fps (list): The frames per second. Default is [60].
"""
data = {
    "camera_index": [1],
    "image_synthesis": 'raytracing',
    "image_width": [1920],
    "image_height": [1080],
    "fps": [60],
}

sio.emit("render", data)

### Change system agent settings
##### You can also enable or disable system agents within the environment. Additional settings such as crowd density, vehicle denisity, parked cars, etc. can be changed via the API

In [ ]:
"""
Args:
    crowd (bool): The crowd data. Default is True.
    traffic (bool): The traffic data. Default is True.
    visualize (bool): Visualize crowd and traffic. Default is False
"""
data = {
    "crowd": True, 
    "traffic": True,
    "visualize": False,
}
sio.emit("system_agent", data)